In [2]:
%connect_info  #prints connection information
%qtconsole  

{
  "stdin_port": 41393, 
  "ip": "127.0.0.1", 
  "control_port": 49714, 
  "hb_port": 55932, 
  "signature_scheme": "hmac-sha256", 
  "key": "b3fdf7d2-2b0e-4043-bab9-f32f58fe6482", 
  "shell_port": 57709, 
  "transport": "tcp", 
  "iopub_port": 45087
}

Paste the above JSON into a file, and connect with:
    $> ipython <app> --existing <file>
or, if you are local, you can connect with just:
    $> ipython <app> --existing /home/obotvinnik/.local/share/jupyter/runtime/kernel-8711ba68-c7ce-49a6-8d33-e6d9799e7b54.json 
or even just:
    $> ipython <app> --existing 
if this is the most recent IPython session you have started.


In [3]:
from IPython.core.display import Javascript
from IPython.display import display

def getname():
    display(Javascript('IPython.notebook.kernel.execute("theNotebook = " + \
    "\'"+IPython.notebook.notebook_name+"\'");'))
    return theNotebook

# Jakob
notebook_name = getname()

<IPython.core.display.Javascript object>

NameError: global name 'theNotebook' is not defined

In [ ]:
figure_folder = 'figures/{}'.format(notebook_name.split('.ipynb')[0])
! mkdir -p $figure_folder
figure_folder

In [ ]:
from __future__ import print_function

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

sns.set(style='white', context='paper', rc={'font.sans-serif':'Arial', 'pdf.fonttype': 42})

In [ ]:
cd /home/obotvinnik/projects/singlecell_pnms/analysis/

In [ ]:
! tree outrigger_v2_bam_unstranded

In [ ]:
! wc -l outrigger_v2_bam/junctions/reads.csv

In [ ]:
! wc -l outrigger_v2/junctions/reads.csv

Why are these different?

In [ ]:
names_to_directories = {'unstranded': 'outrigger_v2_bam_unstranded', 
                        'stranded_bam': 'outrigger_v2_bam', 
                        'stranded': 'outrigger_v2',
                        'stranded_rmdup': 'outrigger_v2_rmdup',
                        'unstranded_rmdup': 'outrigger_v2_rmdup_unstranded'}

In [ ]:
psis = {}

for name, directory in names_to_directories.items():
    psis[name] = pd.read_csv('{}/psi/outrigger_psi.csv'.format(directory), index_col=0)


In [ ]:
dfs = []

for name, directory in names_to_directories.items():
    print(name, directory)
    se = pd.read_csv('{}/psi/se/summary.csv'.format(directory))
    mxe = pd.read_csv('{}/psi/mxe/summary.csv'.format(directory))
    df = pd.concat([se, mxe], ignore_index=True)
    print('\t', df.shape)
    df['sample_id'] = df['sample_id'].map(lambda x: '_'.join(x.split('_')[:2]))
    df['dataset'] = name
    dfs.append(df)
summary = pd.concat(dfs, ignore_index=True)
#     summaries[name] = summary

print(summary.shape)
summary.head()

In [ ]:
%time summary_dropna = summary.groupby(['dataset', 'event_id']).apply(lambda x: x.dropna(subset=['psi'], how='all'))
print(summary_dropna.shape)
summary_dropna.head()

In [ ]:
# %time summary_dropna = summary.groupby(['dataset', 'sample_id', 'event_id']).apply(lambda x: x.dropna(subset=['psi'], how='all'))
# print(summary_dropna.shape)
# summary_dropna.head()

In [ ]:
wide_psi = summary_dropna.pivot_table(index=['sample_id', 'event_id'], columns='dataset', values='psi')
print(wide_psi.shape)
wide_psi = wide_psi.dropna(how='all')
print(wide_psi.shape)
wide_psi.head()

In [ ]:
rps24_event = 'isoform1=junction:chr10:79797063-79800372:+|'\
'isoform2=junction:chr10:79797063-79799961:+@exon:chr10:79799962-79799983:+@junction:chr10:79799984-79800372:+'
rps24_event

In [ ]:
pkm_mxe_event = u'isoform1=junction:chr15:72494962-72499068:-@exon:chr15:72494795-72494961:-@junction:chr15:72492997-72494794:-'\
'|isoform2=junction:chr15:72495530-72499068:-@exon:chr15:72495363-72495529:-@junction:chr15:72492997-72495362:-'
pkm_mxe_event

In [ ]:
idx = pd.IndexSlice

In [ ]:
sample_ids = ['M6_04', 'P1_03']

In [ ]:
wide_psi.loc[idx[:, rps24_event], :].head()

In [ ]:
wide_psi.loc[idx[sample_ids, rps24_event], :]

In [ ]:
summary.query('(sample_id in @sample_ids) & (event_id == @rps24_event)')

In [ ]:
wide_psi.loc[idx[:, pkm_mxe_event], :].head()

In [ ]:
wide_psi.loc[idx[sample_ids, pkm_mxe_event], :]

In [ ]:
summary.query('(sample_id in @sample_ids) & (event_id == @pkm_mxe_event)')

## 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
figure_folder

In [ ]:
! mkdir -p $figure_folder

In [ ]:
g = sns.pairplot(wide_psi.fillna(-0.2), size=2, plot_kws=dict(alpha=0.5, rasterized=True))
g.savefig('{}/stranded_unstranded_rmdup_psi_pairplot.pdf'.format(figure_folder), dpi=300)


In [ ]:
sns.jointplot('unstranded', 'unstranded_rmdup', wide_psi.fillna(-0.2), size=3)

In [ ]:
sns.jointplot('unstranded', 'unstranded_rmdup', wide_psi, size=3)

## Why is stranded bam messed up?

This event is Psi=1 in all

In [ ]:
sample_id, event_id = 'CVN_01', 'isoform1=junction:chr10:102114390-102120490:+|isoform2=junction:chr10:102114390-102116288:+@exon:chr10:102116289-102116521:+@junction:chr10:102116522-102120490:+'

summary.query('(sample_id == @sample_id) & (event_id == @event_id)')

In [ ]:

summary.query('(sample_id == @sample_id) & (event_id == @pkm_mxe_event)')

This event is Psi~1 in all but stranded_bam and stranded_rmdup

In [ ]:
sample_id, event_id = 'CVN_01', 'isoform1=junction:chr10:101507148-101514285:+|isoform2=junction:chr10:101507148-101510125:+@exon:chr10:101510126-101510153:+@junction:chr10:101510154-101514285:+'
summary.query('(sample_id == @sample_id) & (event_id == @event_id)')

Something must have gone wrong when making the junction read file from the `stranded_bam` data for getting the strands because some junctions are NA when they shouldn't be 

## Make Wasabiplots

In [ ]:
from __future__ import print_function

import glob
bam_filenames = glob.glob('/home/obotvinnik/projects/singlecell_pnms/analysis/bams_from_aws/*.bam')

sample_id = 'M2_02'

bam_filename = [x for x in bam_filenames if sample_id in x][0]
bam_filename

In [ ]:
mxe_events = pd.read_csv('outrigger_v2/index/mxe/events.csv')
print(mxe_events.shape)
mxe_events.head()

In [ ]:
mxe_event_bed = pd.read_table('outrigger_v2/index/mxe/event.bed', index_col=3, header=None)
print(mxe_event_bed.shape)
mxe_event_bed.head()

In [ ]:
pkm_mxe_event_bed = mxe_event_bed.loc[pkm_mxe_event]

In [ ]:
pkm_mxe_event_bed.max()

In [ ]:
pkm_mxe_event_bed.min()

In [ ]:
chrom = pkm_mxe_event_bed.iloc[0, 0]
start = pkm_mxe_event_bed.min()[1]
stop = pkm_mxe_event_bed.max()[2]
strand = '-'


In [ ]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd


%matplotlib inline

import flotilla


flotilla_dir = '/projects/ps-yeolab/obotvinnik/flotilla_projects'

study = flotilla.embark('singlecell_pnm_figure2_modalities_bayesian', flotilla_dir=flotilla_dir)

# not_outliers = study.splicing.singles.index.difference(study.splicing.outliers.index)


In [ ]:
import pandas as pd
import os

bam_df = pd.Series(sorted(bam_filenames), name='bam').to_frame()
bam_df['sample_id'] = bam_df['bam'].map(lambda x: '_'.join(os.path.basename(x).split('.')[0].split('_')[:2]))
bam_df = bam_df.set_index('sample_id')
bam_df['phenotype'] = study.sample_id_to_phenotype
bam_df['pooled'] = study.metadata.data['pooled']
bam_df['outlier'] = study.metadata.data['outlier']
bam_df.head()

In [ ]:
import wasabiplot as wp

def wasabifacet(x, **kwargs):
    return wp.wasabiplot(x.iloc[0], **kwargs)

In [ ]:
bam_df_samples = bam_df.loc[sample_ids]
print(bam_df_samples.shape)
bam_df_samples.head()

In [ ]:
%pdb

In [ ]:
g = sns.FacetGrid(bam_df_samples, row='bam', hue='phenotype', size=1.5, aspect=8)
g.map(wasabifacet, 'bam', chrom=chrom, start=start, stop=stop, strand=strand, warn_skipped=False)